In [2]:
import pandas as pd
import numpy as np

# 데이터 불러오기

- Human Labeling된 데이터인 SKT , 현대백화점, 현대모비스 데이터 셋을 이용하여 각 정확도를 올린다.

- 순서
    1. 데이터 불러오기
    2. 데이터 분리
    3. Title 의 토큰화 및 불용어 제거
    4. Label 의 원핫인코딩
    5. 길이가 다른 title의 정형화
    6. LSTM을 통한 딥러닝
    7. 정확도 확인

In [98]:
df_skt = pd.read_csv('../감정분석_V.혜지/title_for_sentiment/total_test.csv')

In [99]:
df_mobis = pd.read_csv('./현대모비스_trainset.csv', encoding = 'ANSI')

In [100]:
df_hb = pd.read_csv('./현대백화점_trainset.csv')
df_hb

,id,Title,label
0,0,"靑 오찬도 없이 간담회 진행…재계 ""외교로 풀 문제를…""",0
1,1,"김정훈 ""공정위 소비자중심경영 인증기업 24%는 행정조치 처분기업""",-1
2,2,"산업부 ""코리아 세일 페스타, 상생의 장터 연다""",1
3,3,"박원순, 선거 마지막 날까지 정순균 강남구청장 후보 지원",0
4,4,"장영철 강남구청장 후보, 선거운동 마지막 주말 총력유세",0
...,...,...,...
216,216,입소문 노리려 인플루언서와 공간 나누는 유통업계(종합),1
217,217,"""인플루언서 모셔라""…'안방' 내주는 유통업계",1
218,218,"현대百, 고메 아이리시 위크…50여개 아일랜드 식품 브랜드 한자리에",1
219,219,풍성하고 즐거운 소비 '경험'…진열방식 바꾸는 패션가(종합),1


In [101]:
df = pd.concat([df_skt, df_mobis, df_hb])

In [102]:
df

,id,Title,label,Column1
0,0.0,국민안전로봇 2021년까지 개발…'로봇물고기' 전철 피할까?,0,NaN
1,1.0,'국정원 해킹 의혹' 상임위 앞두고 與野 전초전,0,NaN
2,2.0,인터넷티비(TV)결합상품 인터넷가입 비교사이트 통해 통신사별 장단점 비교해야 유리,0,NaN
3,3.0,SKT LG KT 인터넷가입 비교사이트 ‘펭귄통신’ 설치 당일 현금지원 이벤트,0,NaN
4,4.0,설 연휴 고향 가는 길 언제 출발해야 덜 막힐까,0,NaN
...,...,...,...,...
216,216.0,입소문 노리려 인플루언서와 공간 나누는 유통업계(종합),1,NaN
217,217.0,"""인플루언서 모셔라""…'안방' 내주는 유통업계",1,NaN
218,218.0,"현대百, 고메 아이리시 위크…50여개 아일랜드 식품 브랜드 한자리에",1,NaN
219,219.0,풍성하고 즐거운 소비 '경험'…진열방식 바꾸는 패션가(종합),1,NaN


In [103]:
from sklearn.model_selection import train_test_split

In [104]:
# Train 데이터와 Test 데이터 분리
X = df['Title']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)

In [105]:
# 문장을 나누기 위한 Stopwords 선정
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

In [106]:
# Sentence Tokenizing

import konlpy
from konlpy.tag import Okt

okt = Okt()
X_train_token = []
for sentence in X_train:
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train_token.append(temp_X)

X_test_token = []
for sentence in X_test:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test_token.append(temp_X)

In [107]:
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences

In [108]:
from keras.preprocessing.text import Tokenizer
max_words = 35000 
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train_token) 
X_train_token = tokenizer.texts_to_sequences(X_train_token) 
X_test_token = tokenizer.texts_to_sequences(X_test_token)

### One-Hot encoding 2가지 방법

In [109]:
# 1. 
import numpy as np
y_train_encode = []
y_test_encode = []
for i in range(len(y_train)):
    if y_train.iloc[i] == 1:
        y_train_encode.append([0, 0, 1]) 
    elif y_train.iloc[i] == 0:
        y_train_encode.append([0, 1, 0]) 
    elif y_train.iloc[i] == -1:
        y_train_encode.append([1, 0, 0]) 
        
for i in range(len(y_test)):
    if y_test.iloc[i] == 1:
        y_test_encode.append([0, 0, 1]) 
    elif y_test.iloc[i] == 0:
        y_test_encode.append([0, 1, 0])
    elif y_test.iloc[i] == -1:
        y_test_encode.append([1, 0, 0]) 
        
y_train_encode=np.array(y_train_encode)
y_test_encode=np.array(y_test_encode)

In [110]:
y_train_encode

array([[0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [111]:
# 2. 
# to_categorical 방법은 0 1 -1 순서로 인코딩된다!
from keras.utils import to_categorical

y_train_cata = to_categorical(y_train, num_classes = 3)
y_test_cata = to_categorical(y_test, num_classes = 3)

In [112]:
y_train_cata

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [113]:
max_len = 20 # pad_sequesces이용, 전체 데이터의 길이를 20로 맞춘다
X_train = pad_sequences(X_train_token, maxlen=max_len)
X_test = pad_sequences(X_test_token, maxlen=max_len)

In [114]:
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy']) 
history = model.fit(X_train, y_train_encode, epochs=10, batch_size=10, validation_split=0.1)

Epoch 1/10
65/65 [==============================] - 5s 73ms/step - loss: 1.0326 - accuracy: 0.4638 - val_loss: 0.9712 - val_accuracy: 0.4932
Epoch 2/10
65/65 [==============================] - 3s 44ms/step - loss: 0.8211 - accuracy: 0.6287 - val_loss: 0.8393 - val_accuracy: 0.6164
Epoch 3/10
65/65 [==============================] - 3s 44ms/step - loss: 0.5232 - accuracy: 0.8028 - val_loss: 1.4024 - val_accuracy: 0.6027
Epoch 4/10
65/65 [==============================] - 3s 43ms/step - loss: 0.3281 - accuracy: 0.9045 - val_loss: 0.8300 - val_accuracy: 0.6575
Epoch 5/10
65/65 [==============================] - 3s 44ms/step - loss: 0.1869 - accuracy: 0.9322 - val_loss: 1.0284 - val_accuracy: 0.6575
Epoch 6/10
65/65 [==============================] - 3s 43ms/step - loss: 0.1314 - accuracy: 0.9630 - val_loss: 0.9582 - val_accuracy: 0.6027
Epoch 7/10
65/65 [==============================] - 3s 44ms/step - loss: 0.0801 - accuracy: 0.9738 - val_loss: 1.0998 - val_accuracy: 0.6438
Epoch 8/10
65

In [115]:
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test_encode)[1]*100))

6/6 [==============================] - 0s 10ms/step - loss: 1.5661 - accuracy: 0.5691

 테스트 정확도 : 56.91%


#### 현대백화점, SKT, 현대모비스를 합한 데이터를 통해 20%의 비율로 Train과 Test를 나누고 감정 라벨링 결과 57%의 정확도를 보인다. 

- parameter를 바꿔서 실행해보기

In [116]:
model2 = Sequential()
model2.add(Embedding(max_words, 100))
model2.add(LSTM(128))
model2.add(Dense(3, activation='softmax'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
history = model2.fit(X_train, y_train_encode, epochs=10, batch_size=10, validation_split=0.1)

Epoch 1/10
65/65 [==============================] - 5s 76ms/step - loss: 1.0286 - accuracy: 0.4607 - val_loss: 0.9940 - val_accuracy: 0.5068
Epoch 2/10
65/65 [==============================] - 4s 55ms/step - loss: 0.7768 - accuracy: 0.6672 - val_loss: 0.7989 - val_accuracy: 0.6712
Epoch 3/10
65/65 [==============================] - 4s 58ms/step - loss: 0.3056 - accuracy: 0.9106 - val_loss: 0.9088 - val_accuracy: 0.6575
Epoch 4/10
65/65 [==============================] - 4s 58ms/step - loss: 0.0835 - accuracy: 0.9676 - val_loss: 1.2671 - val_accuracy: 0.6164
Epoch 5/10
65/65 [==============================] - 4s 58ms/step - loss: 0.0656 - accuracy: 0.9861 - val_loss: 0.9491 - val_accuracy: 0.6301
Epoch 6/10
65/65 [==============================] - 4s 58ms/step - loss: 0.0944 - accuracy: 0.9738 - val_loss: 0.9956 - val_accuracy: 0.5616
Epoch 7/10
65/65 [==============================] - 4s 57ms/step - loss: 0.0590 - accuracy: 0.9846 - val_loss: 0.8810 - val_accuracy: 0.5890
Epoch 8/10
65

In [118]:
print("\n 테스트 정확도 : {:.2f}%".format(model2.evaluate(X_test,y_test_encode)[1]*100))

6/6 [==============================] - 0s 9ms/step - loss: 1.3903 - accuracy: 0.6077

 테스트 정확도 : 60.77%


#### 약  61% !!

In [119]:
# adam 모델 epoch 15로 실행
model2 = Sequential()
model2.add(Embedding(max_words, 100))
model2.add(LSTM(128))
model2.add(Dense(3, activation='softmax'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
history = model2.fit(X_train, y_train_encode, epochs=15, batch_size=10, validation_split=0.1)

Epoch 1/15
65/65 [==============================] - 5s 82ms/step - loss: 1.0292 - accuracy: 0.4422 - val_loss: 0.9746 - val_accuracy: 0.6301
Epoch 2/15
65/65 [==============================] - 4s 55ms/step - loss: 0.7420 - accuracy: 0.6949 - val_loss: 0.7983 - val_accuracy: 0.6027
Epoch 3/15
65/65 [==============================] - 4s 58ms/step - loss: 0.2887 - accuracy: 0.9106 - val_loss: 0.8095 - val_accuracy: 0.6438
Epoch 4/15
65/65 [==============================] - 4s 65ms/step - loss: 0.0845 - accuracy: 0.9753 - val_loss: 0.9432 - val_accuracy: 0.6164
Epoch 5/15
65/65 [==============================] - 4s 57ms/step - loss: 0.0470 - accuracy: 0.9846 - val_loss: 0.8571 - val_accuracy: 0.6575
Epoch 6/15
65/65 [==============================] - 4s 57ms/step - loss: 0.0400 - accuracy: 0.9938 - val_loss: 0.9695 - val_accuracy: 0.5890
Epoch 7/15
65/65 [==============================] - 4s 57ms/step - loss: 0.0180 - accuracy: 0.9938 - val_loss: 1.1330 - val_accuracy: 0.6575
Epoch 8/15
65

In [120]:
print("\n 테스트 정확도 : {:.2f}%".format(model2.evaluate(X_test,y_test_encode)[1]*100))

6/6 [==============================] - 0s 8ms/step - loss: 1.6380 - accuracy: 0.6188

 테스트 정확도 : 61.88%


## Skt ,현대모비스, 현백 데이터를 합쳐서 현대백화점 종목을 추론하기

In [125]:
hb_test = pd.read_csv('./현대백화점_testset.csv', encoding = 'ANSI')
# df = 현백 ,현모, SKT 데이터를 합친것

In [126]:
X_train = df['Title']
X_test = hb_test['Title']

y_train = df['label']
y_test = hb_test['label']

In [127]:
# 문장을 나누기 위한 Stopwords 선정
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

In [128]:
    
# Sentence Tokenizing
import konlpy
from konlpy.tag import Okt

okt = Okt()
X_train_token = []
for sentence in X_train:
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train_token.append(temp_X)

X_test_token = []
for sentence in X_test:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test_token.append(temp_X)
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

max_words = 35000 
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train_token) 
X_train_token = tokenizer.texts_to_sequences(X_train_token) 
X_test_token = tokenizer.texts_to_sequences(X_test_token)

In [132]:
import numpy as np
y_train_encode = []
y_test_encode = []
for i in range(len(y_train)):
    if y_train.iloc[i] == 1:
        y_train_encode.append([0, 0, 1]) 
    elif y_train.iloc[i] == 0:
        y_train_encode.append([0, 1, 0]) 
    elif y_train.iloc[i] == -1:
        y_train_encode.append([1, 0, 0]) 
        
for i in range(len(y_test)):
    if y_test.iloc[i] == 1:
        y_test_encode.append([0, 0, 1]) 
    elif y_test.iloc[i] == 0:
        y_test_encode.append([0, 1, 0])
    elif y_test.iloc[i] == -1:
        y_test_encode.append([1, 0, 0]) 
        
y_train_encode=np.array(y_train_encode)
y_test_encode=np.array(y_test_encode)

In [133]:
max_len = 20 # pad_sequesces이용, 전체 데이터의 길이를 20로 맞춘다
X_train = pad_sequences(X_train_token, maxlen=max_len)
X_test = pad_sequences(X_test_token, maxlen=max_len)

In [134]:
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy']) 
history = model.fit(X_train, y_train_encode, epochs=10, batch_size=10, validation_split=0.1)

Epoch 1/10
82/82 [==============================] - 2s 30ms/step - loss: 1.0177 - accuracy: 0.4852 - val_loss: 1.0752 - val_accuracy: 0.2308
Epoch 2/10
82/82 [==============================] - 2s 22ms/step - loss: 0.8050 - accuracy: 0.6650 - val_loss: 0.6776 - val_accuracy: 0.7253
Epoch 3/10
82/82 [==============================] - 2s 22ms/step - loss: 0.5233 - accuracy: 0.8067 - val_loss: 1.1093 - val_accuracy: 0.4945
Epoch 4/10
82/82 [==============================] - 2s 23ms/step - loss: 0.3335 - accuracy: 0.8904 - val_loss: 0.6651 - val_accuracy: 0.7363
Epoch 5/10
82/82 [==============================] - 2s 24ms/step - loss: 0.1922 - accuracy: 0.9384 - val_loss: 0.9113 - val_accuracy: 0.6374
Epoch 6/10
82/82 [==============================] - 2s 22ms/step - loss: 0.1265 - accuracy: 0.9594 - val_loss: 1.0822 - val_accuracy: 0.5714
Epoch 7/10
82/82 [==============================] - 2s 21ms/step - loss: 0.0820 - accuracy: 0.9741 - val_loss: 0.8833 - val_accuracy: 0.6593
Epoch 8/10
82

In [135]:
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test_encode)[1]*100))

2/2 [==============================] - 0s 2ms/step - loss: 3.3818 - accuracy: 0.5000

 테스트 정확도 : 50.00%


In [136]:
# adam 모델 epoch 15로 실행
model2 = Sequential()
model2.add(Embedding(max_words, 100))
model2.add(LSTM(128))
model2.add(Dense(3, activation='softmax'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
history = model2.fit(X_train, y_train_encode, epochs=15, batch_size=10, validation_split=0.1)

Epoch 1/10
82/82 [==============================] - 4s 53ms/step - loss: 1.0280 - accuracy: 0.4643 - val_loss: 0.7238 - val_accuracy: 0.8681
Epoch 2/10
82/82 [==============================] - 3s 42ms/step - loss: 0.7153 - accuracy: 0.6933 - val_loss: 0.6411 - val_accuracy: 0.7363
Epoch 3/10
82/82 [==============================] - 4s 45ms/step - loss: 0.2450 - accuracy: 0.9187 - val_loss: 0.6358 - val_accuracy: 0.7473
Epoch 4/10
82/82 [==============================] - 3s 41ms/step - loss: 0.0684 - accuracy: 0.9815 - val_loss: 0.9539 - val_accuracy: 0.6484
Epoch 5/10
82/82 [==============================] - 3s 41ms/step - loss: 0.0452 - accuracy: 0.9889 - val_loss: 0.8049 - val_accuracy: 0.6923
Epoch 6/10
82/82 [==============================] - 3s 42ms/step - loss: 0.0214 - accuracy: 0.9963 - val_loss: 0.8072 - val_accuracy: 0.7143
Epoch 7/10
82/82 [==============================] - 3s 41ms/step - loss: 0.0230 - accuracy: 0.9914 - val_loss: 0.8869 - val_accuracy: 0.7143
Epoch 8/10
82

In [137]:
print("\n 테스트 정확도 : {:.2f}%".format(model2.evaluate(X_test,y_test_encode)[1]*100))

2/2 [==============================] - 0s 2ms/step - loss: 3.1094 - accuracy: 0.5476

 테스트 정확도 : 54.76%


In [139]:
history = model2.fit(X_train, y_train_encode, epochs=5, batch_size=5, validation_split=0.1)

Epoch 1/5
163/163 [==============================] - 7s 40ms/step - loss: 0.0064 - accuracy: 0.9988 - val_loss: 0.9033 - val_accuracy: 0.7802
Epoch 2/5
163/163 [==============================] - 7s 41ms/step - loss: 0.0039 - accuracy: 0.9988 - val_loss: 0.8677 - val_accuracy: 0.7912
Epoch 3/5
163/163 [==============================] - 7s 41ms/step - loss: 0.0063 - accuracy: 0.9975 - val_loss: 0.8976 - val_accuracy: 0.7473
Epoch 4/5
163/163 [==============================] - 7s 40ms/step - loss: 0.0031 - accuracy: 0.9975 - val_loss: 0.9184 - val_accuracy: 0.7912
Epoch 5/5
163/163 [==============================] - 7s 41ms/step - loss: 0.0022 - accuracy: 0.9988 - val_loss: 0.9936 - val_accuracy: 0.7802


In [140]:
print("\n 테스트 정확도 : {:.2f}%".format(model2.evaluate(X_test,y_test_encode)[1]*100))

2/2 [==============================] - 0s 3ms/step - loss: 3.1405 - accuracy: 0.5476

 테스트 정확도 : 54.76%
